In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/DLProject/Draft/BeautifyBasedOnGAN')

In [ ]:
# !git clone https://github.com/zoezhou1999/BeautifyBasedOnGAN.git

In [ ]:
%tensorflow_version 1.x
import tensorflow
# print(tensorflow.__version__)
import os
import misc
import numpy as np
import pdb
from config import EasyDict
import tfutil
import argparse

from random import gauss
import matplotlib.pyplot as plt

In [ ]:
tfutil.printtf() # print tensorflow version

1.15.2


In [ ]:
# import tensorflow as tf

# tf.reset_default_graph()
# init=tf.global_variables_initializer()

# sess = tf.InteractiveSession()
# sess.run(init)

# default = tf.get_default_session()
# default == None # True

In [ ]:
# import tensorflow as tf
# tf.reset_default_graph()
# sess = tf.Session()
# print(tf.get_default_session()==None)

In [ ]:
# %tensorflow_version 1.x
dir = '/content/drive/MyDrive/DLProject/Draft/BeautifyBasedOnGAN/beholder-id.pkl'
network_pkl = misc.locate_network_pkl(dir)
print('Loading network from "%s"...' % network_pkl)
G, D, Gs = misc.load_network_pkl(dir, None)

Loading network from "/content/drive/MyDrive/DLProject/Draft/BeautifyBasedOnGAN/beholder-id.pkl"...












/tensorflow-1.15.2/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
type(Gs)

tfutil.Network

In [ ]:
Gs.static_kwargs

{'label_size': 572, 'num_channels': 3, 'resolution': 128}

In [ ]:
Gs.input_shapes

[[None, 512], [None, 572]]

In [ ]:
Gs.num_outputs

1

In [ ]:
# Gs.print_layers()

In [ ]:
def gen_some_img(n_img = 10, start = 0.2, n_levels = 10, step = 0.2):
  imgList = []
  latentList = []
  for j in range(n_img):
    print('Person number = ', j)

    latents = misc.random_latents(1, Gs, random_state=np.random.RandomState(j))

        # if classification asked, perform conditioning using classification vector
    argslabels_size = 60
    np.random.seed(j)
    # vector = [gauss(0, 1) for j in range(512)]
    # mag = sum(x ** 2 for x in vector) ** .5
    # id_vectors=[x/mag for x in vector]
    # id_vectors = np.expand_dims(np.array(id_vectors, dtype=np.float32),axis=0)

    # for i in range(argslabels_size):
    for i in range(n_levels):      
    # initiate conditioned label
      labels = np.ones([1, argslabels_size], np.float32)*(start+i*step)
      # labels = np.random.uniform(start-0.2+i*step,start+0.2+i*step, argslabels_size).reshape(1, -1)
      # if i == 0:
      #   labels = np.random.uniform(0,2, argslabels_size).reshape(1, -1)
      # elif i == 1:
      #   labels = np.random.uniform(1.4,1.6, argslabels_size).reshape(1, -1)
      # else:
      #   labels = np.ones([1, argslabels_size], np.float32)*1.5
      #   for x in range(labels.shape[1]):
      #     if x<30:
      #       labels[0,x] = 0
      #     else:
      #       labels[0,x] = 3

      vector = [gauss(0, 1) for l in range(512)]
      mag = sum(x ** 2 for x in vector) ** .5
      id_vectors=[x/mag for x in vector]
      id_vectors = np.expand_dims(np.array(id_vectors, dtype=np.float32),axis=0)
      combined_labels = np.hstack((labels, id_vectors)).astype(np.float32)
      
      # infer conditioned noise to receive image
      image = Gs.run(latents, combined_labels, minibatch_size=1, num_gpus=1, out_mul=127.5, out_add=127.5, out_shrink=1, out_dtype=np.uint8)
      #image = Gs.run(latents, labels, minibatch_size=1, num_gpus=0, out_mul=127.5, out_add=127.5, out_shrink=1, out_dtype=np.uint8)

      # save generated image as 'i.png' and noise vector as noise_vector.txt
      # misc.save_image_grid(image, os.path.join(result_subdir, '{}_{}.png'.format('%04d' % j,i)), [0,255], [1,1])
      
      # save latent space for later use
      # np.save(os.path.join(result_subdir,'latents_vector.npy'), latents)
      imgList.append(image)
      latentList.append(latents)
      # print('Image number = ', j)
      print('Beauty score = ', ((start+i*step)*4)+1)
      plt.subplots()
      img2 = image[0].transpose(1,2, 0)
      plt.imshow(img2)
      plt.show()
  return imgList, latentList

In [ ]:
import matplotlib.pyplot as plt
def plotimg(imList):
  count = 0
  for image in imList[:10]:
    print('Image number = ', count)
    count += 1
    plt.subplots()
    img2 = image[0].transpose(1,2, 0)
    plt.imshow(img2)
    plt.show()

In [ ]:
imList, latentList = gen_some_img(5, n_levels=5, start = 0, step=0.25)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# network_pkl = misc.locate_network_pkl(args.results_dir)
# print('Loading network from "%s"...' % network_pkl)
# G, D, Gs = misc.load_network_pkl(args.results_dir, None)

# # sample <args.output> sequences
# for j in range(args.outputs):
    
#     # change the random seed in every iteration
#     np.random.seed(j)
    
#     # generate random noise
#     latents = misc.random_latents(1, Gs, random_state=np.random.RandomState(j))
    
#     # if classification asked, perform conditioning using classification vector
#     if args.classification:
#         for i in range(args.labels_size):
            
#             # initiate conditioned label
#             labels = np.zeros([1, args.labels_size], np.float32)
#             labels[0][i] = 1.0

#             vector = [gauss(0, 1) for j in range(512)]
#             mag = sum(x ** 2 for x in vector) ** .5
#             id_vectors=[x/mag for x in vector]
#             id_vectors = np.expand_dims(np.array(id_vectors, dtype=np.float32),axis=0)
#             combined_labels = np.hstack((labels, id_vectors)).astype(np.float32)
            
#             # infer conditioned noise to receive image
#             image = Gs.run(latents, combined_labels, minibatch_size=1, num_gpus=1, out_mul=127.5, out_add=127.5, out_shrink=1, out_dtype=np.uint8)
#             #image = Gs.run(latents, labels, minibatch_size=1, num_gpus=0, out_mul=127.5, out_add=127.5, out_shrink=1, out_dtype=np.uint8)
            
#             # save generated image as 'i.png' and noise vector as noise_vector.txt
#             misc.save_image_grid(image, os.path.join(result_subdir, '{}_{}.png'.format('%04d' % j,i)), [0,255], [1,1])
            
#             # save latent space for later use
#             np.save(os.path.join(result_subdir,'latents_vector.npy'), latents)
            
#     # classification is not asked, we will use varied conditioning vector
#     else:
    
#         min_beauty_level = 1.0 / args.beauty_levels
#         std = min_beauty_level / 2.0 - (min_beauty_level / 10.0)
#         for i in range(args.beauty_levels):
            
#             # initiate beauty rates label
#             if args.total_agreement:
#                 labels = np.ones(args.labels_size)
#                 labels = labels * (min_beauty_level * (i +1))
#                 labels = np.expand_dims(labels, axis=0)
#             else:
#                 labels = np.random.normal(min_beauty_level*(i+1), std, [1, args.labels_size])
#                 labels = np.clip(labels, 0.0, 1.0)
            
#             vector = [gauss(0, 1) for j in range(512)]
#             mag = sum(x ** 2 for x in vector) ** .5
#             id_vectors=[x/mag for x in vector]
#             id_vectors = np.expand_dims(np.array(id_vectors, dtype=np.float32), axis=0)
#             combined_labels = np.hstack((labels, id_vectors)).astype(np.float32)

#             # infer conditioned noise to receive image
#             image = Gs.run(latents, combined_labels, minibatch_size=1, num_gpus=1, out_mul=127.5, out_add=127.5, out_shrink=1, out_dtype=np.uint8)
#             #image = Gs.run(latents, labels, minibatch_size=1, num_gpus=0, out_mul=127.5, out_add=127.5, out_shrink=1, out_dtype=np.uint8)
            
#             # save generated image as 'i.png' and noise vector as noise_vector.txt
#             misc.save_image_grid(image, os.path.join(result_subdir, '{}_{}.png'.format('%04d' % j,i)), [0,255], [1,1])
            
#             # save latent space for later use
#             np.save(os.path.join(result_subdir,'latents_vector.npy'), latents)
    
#     if j % 10 == 0:
#         print("saved {}/{} images".format(j,args.outputs))